In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the "../input/" directory.
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import os
print(os.listdir("../input/fastai-pretrained"))
print(os.listdir("../input"))


In [ ]:
!mkdir -p /tmp/.cache/torch/checkpoints/
!cp ../input/fastai-pretrained/densenet161-8d451a50.pth /tmp/.cache/torch/checkpoints/densenet161-8d451a50.pth
!cp ../input/fastai-pretrained/densenet169-b2777c0a.pth /tmp/.cache/torch/checkpoints/densenet169-b2777c0a.pth

In [ ]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim import lr_scheduler
import time 
import tqdm
from PIL import Image
train_on_gpu = True
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

from collections import OrderedDict
import cv2

import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Dense, Input, Activation, Dropout, GlobalAveragePooling2D, \
    BatchNormalization, concatenate, AveragePooling2D
from keras.optimizers import Adam
import matplotlib.pyplot as plt

from keras import layers
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Model

import keras.backend as K
from keras.models import Sequential

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from torchvision.models import *

In [ ]:
# Loading...
from fastai import *
from fastai.vision import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy as sp
from functools import partial
from sklearn import metrics
from collections import Counter
from fastai.callbacks import *

In [ ]:
## Making pretrained weights work without needing to find the default filename
if not os.path.exists('/tmp/.cache/torch/checkpoints/'):
        os.makedirs('/tmp/.cache/torch/checkpoints/')
!cp '../input/fastai-pretrained/densenet201/densenet201-4c113574.pth' '/tmp/.cache/torch/checkpoints/densenet201-c1103571.pth'

In [ ]:
os.listdir('../input')

In [ ]:
print('Make sure cudnn is enabled:', torch.backends.cudnn.enabled)

In [ ]:
# Set seed fol all
def seed_everything(seed=1358):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything()

In [ ]:
# Reading train_dataset
base_image_dir = os.path.join('..', 'input/humpback-whale-identification/')
base_image_dir2 = os.path.join('..', 'input/new_train_whale/')
train_dir = os.path.join(base_image_dir2,'new_train/')
df = pd.read_csv(os.path.join(base_image_dir, 'train.csv'))
df['path'] = df['Id'].map(lambda x: os.path.join(train_dir,'{}.png'.format(x)))
df = df.drop(columns=['Id'])
df = df.sample(frac=1).reset_index(drop=True) #shuffle dataframe
df.head()

In [ ]:
TRAIN ="../input/humpback-whale-identification/train"
TEST = "../input/humpback-whale-identification/test"
LABELS = "../input/humpback-whale-identification/train.csv"
SUB = "../input/humpback-whale-identification/sample_submission.csv"

def text2number(LABELS):
    df = pd.read_csv(LABELS)
    
    uni_labels = pd.DataFrame(df['Id'].unique(),columns=['Id'])
    uni_labels.index.name = 'ID_new'
    uni_labels = uni_labels.reset_index()


    df = df.merge(uni_labels,on='Id')
    
    return df
    
#submit = [p for _, p, _ in pd.read_csv(SUB).to_records()]
#join = list(tagged.keys()) + submit
df = text2number(LABELS)
df.head()

In [ ]:
tagged = dict([(p, w) for _, p,_, w in df.to_records()])
# tagged.items()

In [ ]:
# Find fluke pics of whales that have less than 5 pics in training data

TRAIN ="../input/humpback-whale-identification/train"
TEST = "../input/humpback-whale-identification/test"
LABELS = "../input/humpback-whale-identification/train.csv"
SUB = "../input/humpback-whale-identification/sample_submission.csv"

train = pd.read_csv(LABELS)
train_count = train.groupby('Id').count().rename(columns={"Image":"image_count"})
train = train.merge(train_count,on=['Id'])
filelist = train['Image'].loc[(train['image_count']>10)].tolist()
filelist[:10]

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# from scipy import misc, ndimage
# import keras
# from keras import backend as k
# from keras.preprocessing.image import ImageDataGenerator
# import matplotlib.image as mpimg
# from time import time
# import tqdm 

# path = "../input/humpback-whale-identification/train"       
# gen = ImageDataGenerator(rotation_range=20, 
#                          width_shift_range= 0.1, 
#                          height_shift_range = 0.1)

# tagged_new = tagged.copy() 

# t = time()          
# for img in tqdm(filelist):
#     try:
#         class_num = tagged[img]
#         image = np.expand_dims(mpimg.imread(os.path.join(path,img)),0)  
#         aug_iter = gen.flow(image)
#         aug_images = [next(aug_iter)[0].astype(np.uint8) for i in range(5)]
#         aug_image_names = [str(i)+"_"+img for i in range(5)]
#         for i in range(5):
#             plt.imsave(os.path.join(path,aug_image_names[i]),aug_images[i])
#             tagged_new[aug_image_names[i]] = class_num
#     except:
#         pass
# print(time()-t)

In [ ]:
# len(tagged_new)

In [ ]:
# Set Batch Size and Image size
bs = 32 
sz=100

In [ ]:
# value = {}
# count = 0
# for whale in df['p.unique():
#     value[whale] = count
#     count += 1

In [ ]:
filelist[:5]

In [ ]:
# df['keep'] = df['Id']

def keep(data):
    if data in filelist:
        return 1
    else:
        return 0
    
df['keep'] = df['Image'].map(keep)
df['keep'].value_counts()

In [ ]:
df_5 = df[df['keep'] == 1]


In [ ]:
df_small = df_5[2500:5000]
df_new_whale = df_5[df_5['Id'] == 'new_whale']
df_not_new = df_5[df_5['Id'] != 'new_whale']
print(len(df_new_whale), len(df_not_new))

df_1 = df_new_whale[:500]
df_2 = df_not_new[:2000]
df_training = df_1.append(df_2)

In [ ]:
df_1['Image'].value_counts()

In [ ]:
# run this chunk only when augmented data is used
df_new = pd.DataFrame({'Image':list(tagged_new.keys()),
                       'ID_new':list(tagged_new.values())})
df_new = df_new.merge(df[['ID_new','Id']].drop_duplicates(),how='left',on='ID_new')
print(len(df_new))
df_new.head()

In [ ]:
PATH = Path('../input/humpback-whale-identification/')
PATH2 = Path('../input/new_train_whale/')

df_train = pd.read_csv(PATH/'train.csv')


In [ ]:
data = ImageDataBunch.from_df(df=df_training,
                              path=PATH, folder='train', 
                              valid_pct=0.1,
                              ds_tfms=get_transforms(flip_vert=True, max_warp=0.1, max_zoom=1.15, max_rotate=45.),
                              size=224,
                              bs=50, 
                              num_workers=os.cpu_count()
                             ).normalize(imagenet_stats)

In [ ]:
print(f'Classes: {data.classes}')


In [ ]:
data.show_batch(rows=3, figsize=(7,6))

In [ ]:
# Making pretrained weights work without needing to find the default filename
if not os.path.exists('/tmp/.cache/torch/checkpoints/'):
        os.makedirs('/tmp/.cache/torch/checkpoints/')
!cp '../input/fastai-pretrained/resnet50-19c8e357.pth' '/tmp/.cache/torch/checkpoints/resnet50-19c8e357.pth'
!cp '../input/fastai-pretrained/densenet161-8d451a50.pth' '/tmp/.cache/torch/checkpoints/densenet161-8d451a50.pth'

In [ ]:
!cp '../input/fastai-pretrained/densenet201-c1103571.pth' '/tmp/.cache/torch/checkpoints/densenet201-c1103571.pth'

In [ ]:
# PATH = "../input/aptos2019-blindness-detection"
!mkdir -p /tmp/.cache/torch/checkpoints/
!cp ../input/fastai-pretrained/densenet161-8d451a50.pth /tmp/.cache/torch/checkpoints/densenet161-8d451a50.pth
!cp ../input/fastai-pretrained/densenet169-b2777c0a.pth /tmp/.cache/torch/checkpoints/densenet169-b2777c0a.pth
!cp ../input/fastai-pretrained/densenet169-b2777c0a.pth /tmp/.cache/torch/checkpoints/densenet169-b2777c0a.pth
!cp ../input/fastai-pretrained/densenet201-c1103571.pth /tmp/.cache/torch/checkpoints/densenet201-c1103571.pth
# densenet201-c1103571.pth


In [ ]:
learn_1 = cnn_learner(data, models.densenet201, metrics=[accuracy], pretrained=True).mixup()

In [ ]:
learn_1.fit_one_cycle(10,1e-02)
learn_1.recorder.plot_losses()
learn_1.recorder.plot_metrics()

In [ ]:
learn_1.unfreeze()
learn_1.fit_one_cycle(4, slice(5e-6, 0.001/5))

In [ ]:
# learn_2.fit_one_cycle(10,1e-02)
learn_1.recorder.plot_losses()
learn_1.recorder.plot_metrics()

In [ ]:
learn_1.freeze()
learn_1.fit_one_cycle(4, slice(5e-6, 0.001/5))
learn_1.recorder.plot_losses()
learn_1.recorder.plot_metrics()

In [ ]:
learn_1.unfreeze()
learn_1.fit_one_cycle(4, slice(5e-6, 0.001/5))
learn_1.recorder.plot_losses()
learn_1.recorder.plot_metrics()

In [ ]:
learn_1.model

In [ ]:
learn_1.validate(metrics=[accuracy])
# learn.lr_find()

In [ ]:
learn_1.validate.plot_metrics()